# **Ejemplo de uso del framework Stable Baselines 3 en Space Invaders**

Primero instalamos el framework

In [2]:
# Esto hacía falta antes
#!pip install --upgrade --force-reinstall gym

# La opcion oficial no funciona 5/4/2023
#!pip install stable_baselines3[extra]

# Alternativa 
!pip install git+https://github.com/carlosluis/stable-baselines3@fix_tests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/carlosluis/stable-baselines3 (to revision fix_tests) to /tmp/pip-req-build-8_y9ub8r
  Running command git clone --filter=blob:none --quiet https://github.com/carlosluis/stable-baselines3 /tmp/pip-req-build-8_y9ub8r
  Running command git checkout -b fix_tests --track origin/fix_tests
  Switched to a new branch 'fix_tests'
  Branch 'fix_tests' set up to track remote branch 'fix_tests' from 'origin'.
  Resolved https://github.com/carlosluis/stable-baselines3 to commit 6617e6e73cb3a70f3e88cea780ea12bed95c099e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


descargamos los roms de atari

In [3]:
!pip3 install gym[atari,accept-rom-license]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Importamos las librerias necesarias.

In [4]:
import gym
#from gym.wrappers import Monitor

In [5]:
import base64
from pathlib import Path
from IPython import display as ipythondisplay
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
import os

from stable_baselines3.common.evaluation import evaluate_policy

Cargamos el entorno

In [ ]:
ENV_NAME = 'SpaceInvaders-v4'
env = gym.make(ENV_NAME)
height, width, channels = env.observation_space.shape
actions = env.action_space.n

Preparamos el entorno para poder descargar los videos

In [ ]:
# experiment_dir = os.path.abspath("./experiments/{}".format(env.spec.id))
# monitor_path = os.path.join(experiment_dir, "monitor")

# if not os.path.exists(monitor_path):
#     os.makedirs(monitor_path)
# env = Monitor(env, directory=monitor_path, video_callable=lambda count: count % record_video_every == 0, resume=True)
# record_video_every =1



Cargamos el modelo precargado, le damos a entrenar y luego salvamos el entorno.

Vamos a dejar todos los parametros por defecto, que en el caso del Advantage Actor Critic (A2C) usa:
- learning_rate = 0,0007 , 
- n_steps = 5 , 
- gamma = 0,99 ,
- gae_lambda = 1,0 , 
- ent_coef = 0,0 , 
- vf_coef = 0,5 , 
- max_grad_norm = 0,5 , 
- rms_prop_eps = 1e-05

In [ ]:
model = A2C("MlpPolicy", env, verbose=1) #Podemos elegir entre varias politicas, como MLPpolicy, CnnPolicy, MultiInputPolicy
model.learn(total_timesteps=5000)
model.save("a2c_SpaceInvaders")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
-------------------------------------
| time/                 |           |
|    fps                | 36        |
|    iterations         | 100       |
|    time_elapsed       | 13        |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -1.51     |
|    explained_variance | 0.0363    |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -6.86e-05 |
|    value_loss         | 1.52e-09  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 514       |
|    ep_rew_mean        | 50        |
| time/                 |           |
|    fps                | 35        |
|    iterations         | 200       |
|    time_elapsed       | 27        |
|    total_timesteps    | 100

In [ ]:
recompensa_media, desviacion = evaluate_policy(model, model.get_env(), deterministic=True, n_eval_episodes=20)

In [ ]:
print(f"recompensa_media:{recompensa_media:.2f} +/- {desviacion:.2f}")

recompensa_media:0.00 +/- 0.00


--------------------
# Implementación de DQL


Para implementar _Deep Q-Network_, necesitaremos emplear la biblioteca _TensorFlow_.

In [1]:
!pip3 install tensorflow==2.10.1
!pip install keras-rl2
!pip install h5py
!pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
print("The size of frame is: ", env.observation_space.shape)
print("No. of Actions: ", env.action_space.n)

The size of frame is:  (210, 160, 3)
No. of Actions:  6


## Preprocesamiento de los datos

El preprocesado es un paso importante, ya que buscamos reducir la complejidad de nuestros estados para reducir el tiempo de computación necesari durante el entrenamiento.

Los pasos que vamos a seguir son:

* Pasar cada uno de los frames a escala de grises (porque el color no añade información relevante).

* Recortar la pantalla (en nuestro caso, eliminamos la parte de debajo del jugador porque no aporta información útil).

* Normalizar los valores de los píxeles.

* Redimensionar el frame procesado.

In [24]:
!pip install -U scikit-image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 12.1 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
  Attempting uninstall: pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.19.3
    Uninstalling scikit-image-0.19.3:
      Successfully uninstalled scikit-image-0.19.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dop

In [32]:
from skimage import transform
from skimage import io
from skimage.color import rgb2gray
import tensorflow as tf
import matplotlib.pyplot as plt

from collections import deque

import random

import warnings # This ignore all the warning messages that are normally printed during the training because of skiimage
warnings.filterwarnings('ignore') 

In [27]:
"""
    preprocess_frame:
    Take a frame.
    Grayscale it
    Resize it.
        __________________
        |                 |
        |                 |
        |                 |
        |                 |
        |_________________|
        
        to
        _____________
        |            |
        |            |
        |            |
        |____________|
    Normalize it.
    
    return preprocessed_frame
    
    """
def preprocess_frame(frame):
    # Greyscale frame 
    gray = rgb2gray(frame)
    
    # Crop the screen (remove the part below the player)
    # [Up: Down, Left: right]
    cropped_frame = gray[8:-12,4:-12]
    
    # Normalize Pixel Values
    normalized_frame = cropped_frame/255.0
    
    # Resize
    preprocessed_frame = transform.resize(normalized_frame, [110,84])
    
    return preprocessed_frame # 110x84x1 frame

Vamos a apilar (_stack_) los frames.
Apilar los frames es realmente importante porque nos ayuda a darle sentido de movimiento a nuestra Red Neuronal.

PERO, no apilamos cada frame, saltamos 4 frames en cada paso de tiempo. Esto significa que solo se considera cada cuatro frames. Luego, usamos este frame para formar el frame apilado.

El método de salto de frames ya está implementado en la biblioteca.

* Primero preprocesamos el frame.

* Luego lo añadimos a la cola (deque) que automáticamente elimina el frame más antiguo.

* Finalmente construimos el estado apilado.

Así es como funciona el apilado:

* Para el primer frame, alimentamos con 4 frames.
* En cada paso de tiempo, añadimos el nuevo frame a la cola y luego los apilamos para formar un nuevo frame apilado.

Y así sucesivamente.

**Si hemos terminado, creamos una nueva pila con 4 frames nuevos (porque estamos en un nuevo episodio).**

In [28]:
stack_size = 4 # We stack 4 frames

# Initialize deque with zero-images one array for each image
stacked_frames  =  deque([np.zeros((110,84), dtype=np.int) for i in range(stack_size)], maxlen=4)

def stack_frames(stacked_frames, state, is_new_episode):
    # Preprocess frame
    frame = preprocess_frame(state)
    
    if is_new_episode:
        # Clear our stacked_frames
        stacked_frames = deque([np.zeros((110,84), dtype=np.int) for i in range(stack_size)], maxlen=4)
        
        # Because we're in a new episode, copy the same frame 4x
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        stacked_frames.append(frame)
        
        # Stack the frames
        stacked_state = np.stack(stacked_frames, axis=2)
        
    else:
        # Append frame to deque, automatically removes the oldest frame
        stacked_frames.append(frame)

        # Build the stacked state (first dimension specifies different frames)
        stacked_state = np.stack(stacked_frames, axis=2) 
    
    return stacked_state, stacked_frames

In [29]:
### MODEL HYPERPARAMETERS
state_size = [110, 84, 4]      # Our input is a stack of 4 frames hence 110x84x4 (Width, height, channels) 
action_size = env.action_space.n # 8 possible actions
learning_rate =  0.00025      # Alpha (aka learning rate)

### TRAINING HYPERPARAMETERS
total_episodes = 50            # Total episodes for training
max_steps = 50000              # Max possible steps in an episode
batch_size = 64                # Batch size

# Exploration parameters for epsilon greedy strategy
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability 
decay_rate = 0.00001           # exponential decay rate for exploration prob

# Q learning hyperparameters
gamma = 0.9                    # Discounting rate

### MEMORY HYPERPARAMETERS
pretrain_length = batch_size   # Number of experiences stored in the Memory when initialized for the first time
memory_size = 1000000          # Number of experiences the Memory can keep

### PREPROCESSING HYPERPARAMETERS
stack_size = 4                 # Number of frames stacked

### MODIFY THIS TO FALSE IF YOU JUST WANT TO SEE THE TRAINED AGENT
training = False

## TURN THIS TO TRUE IF YOU WANT TO RENDER THE ENVIRONMENT
episode_render = False

Ahora vamos a crear el modelo DQL:
- Tomamos una pila de 4 fotogramas como entrada.
- Pasa a través de 3 convnets.
- Luego se aplana.
- Finalmente pasa a través de 2 capas completamente conectadas (_dense_).
- Devuelve un valor Q para cada acción.

In [47]:
import tensorflow as tf

class DQNetwork:
    def __init__(self, state_size, action_size, learning_rate, name='DQNetwork'):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        
        with tf.name_scope(name):
            # We create the placeholders
            # *state_size means that we take each elements of state_size in tuple hence is like if we wrote
            # [None, 84, 84, 4]
            self.inputs_ = tf.placeholder(tf.float32, [None, *state_size], name="inputs")
            self.actions_ = tf.placeholder(tf.float32, [None, self.action_size], name="actions_")
            
            # Remember that target_Q is the R(s,a) + ymax Qhat(s', a')
            self.target_Q = tf.placeholder(tf.float32, [None], name="target")
            
            """
            First convnet:
            CNN
            ELU
            """
            # Input is 110x84x4 
            self.conv1 = tf.keras.layers.Conv2D(filters=32,
                                  kernel_size=[8, 8],
                                  strides=[4, 4],
                                  padding="VALID",
                                  kernel_initializer=tf.keras.initializers.GlorotUniform(),
                                  name="conv1")(self.inputs_)

            
            self.conv1_out = tf.nn.elu(self.conv1, name="conv1_out")
            
            """
            Second convnet:
            CNN
            ELU
            """
            self.conv2 = tf.keras.layers.Conv2D(filters = 64,
                                 kernel_size = [4,4],
                                 strides = [2,2],
                                 padding = "VALID",
                                kernel_initializer=tf.keras.initializers.GlorotUniform(),
                                 name = "conv2")(self.conv1_out)

            self.conv2_out = tf.nn.elu(self.conv2, name="conv2_out")            
            
            """
            Third convnet:
            CNN
            ELU
            """
            self.conv3 = tf.keras.layers.Conv2D(filters = 64,
                                 kernel_size = [3,3],
                                 strides = [2,2],
                                 padding = "VALID",
                                kernel_initializer=tf.keras.initializers.GlorotUniform(),
                                 name = "conv3")(self.conv2_out)

            self.conv3_out = tf.nn.elu(self.conv3, name="conv3_out")
            
            self.flatten = tf.keras.layers.Flatten()(self.conv3_out)
            
            self.fc1 = tf.keras.layers.Dense(units = 512,
                                  activation = tf.nn.elu,
                                  kernel_initializer=tf.keras.initializers.GlorotUniform(),
                                  name="fc1")(self.flatten)
            
            self.output = tf.keras.layers.Dense(units = self.action_size, 
                                           activation=None,
                                           kernel_initializer=tf.keras.initializers.GlorotUniform())(self.fc1)
            

  
            # Q is our predicted Q value.
            self.Q = tf.reduce_sum(tf.multiply(self.output, self.actions_))
            
            # The loss is the difference between our predicted Q_values and the Q_target
            # Sum(Qtarget - Q)^2
            self.loss = tf.reduce_mean(tf.square(self.target_Q - self.Q))
            
            self.optimizer = tf.keras.optimizers.Adam(self.learning_rate).minimize(self.loss)



In [48]:
# Instantiate the DQNetwork
DQNetwork = DQNetwork(state_size, action_size, learning_rate)

AttributeError: ignored

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import relu, linear
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
import gym
import numpy as np

# Obtener el entorno y extraer el número de acciones
ENV_NAME = 'SpaceInvaders-v4'
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n

# Construir el modelo con capas convolucionales
model = Sequential()
model.add(Conv2D(16, (3, 3), activation=relu, input_shape= env.observation_space.shape))
model.add(Conv2D(32, (3, 3), activation=relu))
model.add(Flatten())
model.add(Dense(nb_actions, activation=linear))
print(model.summary())

# Configurar y compilar el agente
memory = SequentialMemory(limit=5000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 208, 158, 16)      448       
                                                                 
 conv2d_5 (Conv2D)           (None, 206, 156, 32)      4640      
                                                                 
 flatten_2 (Flatten)         (None, 1028352)           0         
                                                                 
 dense_2 (Dense)             (None, 6)                 6170118   
                                                                 
Total params: 6,175,206
Trainable params: 6,175,206
Non-trainable params: 0
_________________________________________________________________
None


In [20]:
print(env.observation_space.shape)
print(type(env))

(210, 160, 3)
<class 'gym.wrappers.order_enforcing.OrderEnforcing'>


In [12]:
# Modificar la entrada para tener 4 dimensiones
#obs = np.expand_dims(env.reset(), axis=0)
dqn.fit(env, nb_steps=5000, visualize=False, verbose=2, nb_max_episode_steps=10000)

# After training is done, we save the final weights.
dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

Training for 5000 steps ...


ValueError: ignored